# Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os
path= '/content/drive/MyDrive/codigos/Anal-tica-en-Recursos-Humanos'
os.chdir(path)
sys.path.append(path+'/')

# Visual Studio

In [16]:
import os
import sys

ruta =os.getcwd()
sys.path.append(ruta)

# Despliegue

In [23]:
import a_funciones as fn ### importar modulo de  funciones
from a_funciones import * ### importar toddas llas funciones del archivo
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import joblib


import numpy as np

El despliegue consiste en dejar todo el código listo para una ejecucion automática en el periodo definido:
En este caso se ejecutara el proceso de entrenamiento y prediccion anualmente.

In [24]:
#### conectarse a base de datos preprocesada

con=sql.connect('bases de datos RRHH/datos.db')
cur=con.cursor()

In [25]:
#crear dataframe para el modelado(2015)
df = pd.read_sql("""select * from completo
where InfoDate='2016-12-31'
""",con=con)

In [27]:
# Hacer la preparacion de los datos nuevos
df_t= fn.preparar_datos(df)

KeyError: "None of [Index(['DepID', 'GenderID', 'level2', 'MaritalDesc', 'FromDiversityJobFairID',\n       'position', 'State', 'CitizenDesc', 'HispanicLatino', 'RaceDesc',\n       'RecruitmentSource'],\n      dtype='object')] are in the [columns]"

In [ ]:
# Cargar modelo entrenado
rf_final = joblib.load("salidas\\rf_final.pkl")

In [ ]:
# Realizar predicciones
predicciones = rf_final.predict(df_t)
pd_pred = pd.DataFrame(predicciones, columns=['Atrition'])

# Crear DataFrame con predicciones
perf_pred = pd.concat([df['EmployeeID'], df_t, pd_pred], axis=1)

# Guardar predicciones en archivos
perf_pred[['EmployeeID', 'Atrition']].to_excel("salidas\\prediccion.xlsx")
#Guardar importancia de las caracteristicas a la hora de predecir
feature_names = df_t.columns


In [ ]:
importances = pd.DataFrame({'Feature': feature_names, 'Importance': rf_final.feature_importances_})
importances.to_excel("salidas\\importances.xlsx")

In [ ]:
# Ver las 10 predicciones más bajas
emp_pred_bajo = perf_pred.sort_values(by=["Atrition"], ascending=True).head(10)
emp_pred_bajo.set_index('EmployeeID', inplace=True)
pred = emp_pred_bajo.T
print(pred)